In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

2023-09-26 11:09:00.389632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 11:09:00.913237: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
batch_size = 2048
l_rate = 0.001

In [4]:
@tf.keras.saving.register_keras_serializable()
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=128, activation=tf.nn.leaky_relu)
        self.dense4 = tf.keras.layers.Dense(units=1024, activation=tf.nn.leaky_relu)
        self.dense5 = tf.keras.layers.Dense(units=8)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        output = self.dense5(x)
        return output

In [5]:
def valiAll(index_epoch):
    y_v_p = model(X_v)
    va_mse = tf.reduce_mean(tf.square(y_v_p - y_v))
    va_rmse = tf.sqrt(va_mse)
    va_mae = tf.reduce_mean(tf.abs(y_v_p - y_v))
    va_r2 = 1 - tf.reduce_sum(tf.square(y_v_p - y_v)) / tf.reduce_sum(tf.square(y_v - tf.reduce_mean(y_v)))
    print("mse:{} rmse:{} mae:{} r2:{}".format(va_mse, va_rmse, va_mae, va_r2))

In [6]:
test_dataset = pd.read_csv('testset.csv', encoding='utf-8').sample(frac=1).reset_index(drop=True)
X_v = test_dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y_v = test_dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)

In [19]:
dataset1 = pd.read_csv('./20-24Trainset.csv', encoding='utf-8')
dataset2 = pd.read_csv('./50-54Trainset.csv', encoding='utf-8')
dataset = pd.concat([dataset1, dataset2], axis=0).sample(frac=1).reset_index(drop=True)
# dataset = dataset[dataset['freq'].isin([2.0, 5.0])]
# dataset = dataset[dataset['freq'].isin([2.1, 5.1])]
# dataset = dataset[dataset['freq'].isin([2.2, 5.2])]
# dataset = dataset[dataset['freq'].isin([2.3, 5.3])]
dataset = dataset[dataset['freq'].isin([2.4, 5.4])]
X = dataset.loc[:,'freq':'L2'].to_numpy(dtype = np.float32)
y = dataset.loc[:,'S11r':'S41i'].to_numpy(dtype = np.float32)
dataset_train = tf.data.Dataset.from_tensor_slices((X, y))
dataset_train = dataset_train.shuffle(buffer_size=X.shape[0])
dataset_train = dataset_train.batch(batch_size)
dataset_train = dataset_train.prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate)

In [21]:
for i in range(150):
    for X, y in dataset_train:
        with tf.GradientTape() as tape:
            y_pred = model(X)
            tr_mse = tf.reduce_mean(tf.square(y_pred - y))
        tr_rmse = tf.sqrt(tr_mse)
        tr_mae = tf.reduce_mean(tf.abs(y_pred - y))
        tr_r2 = 1 - tf.reduce_sum(tf.square(y_pred - y)) / tf.reduce_sum(tf.square(y - tf.reduce_mean(y)))
        grads = tape.gradient(tr_mse, model.variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    print("epoch:{}".format(i))
    print("train mse:{} rmse:{} mae:{} r2:{}".format(tr_mse, tr_rmse, tr_mae, tr_r2))
    valiAll(i)

epoch:0
train mse:0.08370240777730942 rmse:0.2893136739730835 mae:0.22583341598510742 r2:0.31617313623428345
mse:0.08863778412342072 rmse:0.2977209687232971 mae:0.23332378268241882 r2:0.27403098344802856
epoch:1
train mse:0.07809446007013321 rmse:0.2794538736343384 mae:0.21840858459472656 r2:0.36221593618392944
mse:0.07906721532344818 rmse:0.2811889350414276 mae:0.21953335404396057 r2:0.3524166941642761
epoch:2
train mse:0.07073592394590378 rmse:0.26596224308013916 mae:0.20655274391174316 r2:0.4222410321235657
mse:0.07639298588037491 rmse:0.27639278769493103 mae:0.21579396724700928 r2:0.3743193745613098
epoch:3
train mse:0.06833530217409134 rmse:0.2614102065563202 mae:0.2038770169019699 r2:0.44046831130981445
mse:0.075298972427845 rmse:0.2744065821170807 mae:0.2135288566350937 r2:0.38327962160110474
epoch:4
train mse:0.06597308814525604 rmse:0.25685226917266846 mae:0.19760048389434814 r2:0.4606260061264038
mse:0.06981787085533142 rmse:0.2642306983470917 mae:0.20524398982524872 r2:0.428